<a href="https://colab.research.google.com/github/devashree1923/Chat_Bot/blob/main/Embeddings_Abstractive_Summ_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!tar -xvf '/content/gdrive/MyDrive/gl_100d_oa_cr.tar.gz'

GL_100/
GL_100/converted_to_word2vec_100.txt


In [4]:
glove_path = '/content/GL_100/converted_to_word2vec_100.txt'

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

path = "/content/gdrive/MyDrive/biorxiv_clean.csv"
df = pd.read_csv(path, nrows = 100)
df = df.dropna().reset_index()
df = df.drop(columns =['text','index', 'paper_id', 'authors', 'affiliations', 'bibliography', 'raw_authors', 'raw_bibliography'], axis = 1)
df.head()

,title,abstract
0,The Effectiveness of Targeted Quarantine for M...,Abstract\n\nWe model the extent to which age t...
1,Evaluation of Group Testing for SARS-CoV-2 RNA,Abstract\n\nDuring the current COVID-19 pandem...
2,Why estimating population-based case fatality ...,Abstract\n\nDifferent ways of calculating mort...
3,Far-UVC light: A new tool to control the sprea...,Abstract\n\nAirborne-mediated microbial diseas...
4,Quantifying SARS-CoV-2 transmission suggests e...,Abstract\n\nThe newly emergent human virus SAR...


In [7]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"title":"target_text", "abstract":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: Abstract\n\nWe model the extent to ...,The Effectiveness of Targeted Quarantine for M...
1,summarize: Abstract\n\nDuring the current COVI...,Evaluation of Group Testing for SARS-CoV-2 RNA
2,summarize: Abstract\n\nDifferent ways of calcu...,Why estimating population-based case fatality ...
3,summarize: Abstract\n\nAirborne-mediated micro...,Far-UVC light: A new tool to control the sprea...
4,summarize: Abstract\n\nThe newly emergent huma...,Quantifying SARS-CoV-2 transmission suggests e...
...,...,...
78,summarize: Abstract\n\nWe utilized a cell-leve...,Connectivity analyses of bioenergetic changes ...
79,summarize: Abstract\n\nBackground: Imagery tec...,Clinical characterization and chest CT finding...
80,summarize: Abstract\n\nMost models of the COVI...,Mitigating COVID-19 outbreak via high testing ...
81,summarize: Abstract\n\nAfter the outbreak of t...,Clinical Characteristics of Two Human to Human...


In [8]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((66, 2), (17, 2))

In [9]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    T5ForConditionalGeneration,
    MT5ForConditionalGeneration,
    ByT5Tokenizer,
    PreTrainedTokenizer,
    T5TokenizerFast as T5Tokenizer,
    MT5TokenizerFast as MT5Tokenizer,
    AutoTokenizer, 
    LongT5ForConditionalGeneration
)
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelWithLMHead, AutoTokenizer
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar

torch.cuda.empty_cache()
pl.seed_everything(42)


class PyTorchDataModule(Dataset):
    """  PyTorch Dataset class  """

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        source_max_token_len: int = 1024,
        target_max_token_len: int = 512,
    ):
        """
        initiates a PyTorch Dataset Module for input data
        Args:
            data (pd.DataFrame): input pandas dataframe. Dataframe must have 2 column --> "source_text" and "target_text"
            tokenizer (PreTrainedTokenizer): a PreTrainedTokenizer (T5Tokenizer, MT5Tokenizer, or ByT5Tokenizer)
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        """ returns length of data """
        return len(self.data)

    def __getitem__(self, index: int):
        """ returns dictionary of input tensors to feed into T5/MT5 model"""

        data_row = self.data.iloc[index]
        source_text = data_row["source_text"]

        source_text_encoding = self.tokenizer(
            source_text,
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        target_text_encoding = self.tokenizer(
            data_row["target_text"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        labels = target_text_encoding["input_ids"]
        labels[
            labels == 0
        ] = -100  # to make sure we have correct labels for T5 text generation

        return dict(
            source_text_input_ids=source_text_encoding["input_ids"].flatten(),
            source_text_attention_mask=source_text_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=target_text_encoding["attention_mask"].flatten(),
        )


class LightningDataModule(pl.LightningDataModule):
    """ PyTorch Lightning data class """

    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = 4,
        source_max_token_len: int = 1024,
        target_max_token_len: int = 512,
        num_workers: int = 2,
    ):
        """
        initiates a PyTorch Lightning Data Module
        Args:
            train_df (pd.DataFrame): training dataframe. Dataframe must contain 2 columns --> "source_text" & "target_text"
            test_df (pd.DataFrame): validation dataframe. Dataframe must contain 2 columns --> "source_text" & "target_text"
            tokenizer (PreTrainedTokenizer): PreTrainedTokenizer (T5Tokenizer, MT5Tokenizer, or ByT5Tokenizer)
            batch_size (int, optional): batch size. Defaults to 4.
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
        """
        super().__init__()

        self.train_df = train_df
        self.test_df = test_df
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len
        self.num_workers = num_workers

    def setup(self, stage=None):
        self.train_dataset = PyTorchDataModule(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len,
        )
        self.test_dataset = PyTorchDataModule(
            self.test_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len,
        )

    def train_dataloader(self):
        """ training dataloader """
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        """ test dataloader """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        """ validation dataloader """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )


class LightningModel(pl.LightningModule):
    """ PyTorch Lightning Model class"""

    def __init__(
        self,
        tokenizer,
        model,
        outputdir: str = "outputs",
        save_only_last_epoch: bool = False,
    ):
        """
        initiates a PyTorch Lightning Model
        Args:
            tokenizer : T5/MT5/ByT5 tokenizer
            model : T5/MT5/ByT5 model
            outputdir (str, optional): output directory to save model checkpoints. Defaults to "outputs".
            save_only_last_epoch (bool, optional): If True, save just the last epoch else models are saved for every epoch
        """
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.outputdir = outputdir
        self.average_training_loss = None
        self.average_validation_loss = None
        self.save_only_last_epoch = save_only_last_epoch

    def forward(self,  attention_mask, inputs_embeds, decoder_attention_mask, labels=None):
        """ forward step """
        output = self.model(
            # input_ids,
            attention_mask=attention_mask,
            inputs_embeds = inputs_embeds,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask,
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_size):
        """ training step """
        input_ids = batch["source_text_input_ids"]
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            # input_ids = input_ids,
            attention_mask=attention_mask,
            inputs_embeds = inputs_embeds,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log(
            "train_loss", loss, prog_bar=True, logger=True, on_epoch=True, on_step=True
        )
        return loss

    def validation_step(self, batch, batch_size):
        """ validation step """
        input_ids = batch["source_text_input_ids"]
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            # input_ids = input_ids,
            attention_mask=attention_mask,
            inputs_embeds = inputs_embeds,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log(
            "val_loss", loss, prog_bar=True, logger=True, on_epoch=True, on_step=True
        )
        return loss

    def test_step(self, batch, batch_size):
        """ test step """
        input_ids = batch["source_text_input_ids"]
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            # input_ids = input_ids,
            attention_mask=attention_mask,
            inputs_embeds = inputs_embeds,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        """ configure optimizers """
        return AdamW(self.parameters(), lr=0.0001)

    def training_epoch_end(self, training_step_outputs):
        """ save tokenizer and model on epoch end """
        self.average_training_loss = np.round(
            torch.mean(torch.stack([x["loss"] for x in training_step_outputs])).item(),
            4,
        )
        path = f"{self.outputdir}/simplet5-epoch-{self.current_epoch}-train-loss-{str(self.average_training_loss)}-val-loss-{str(self.average_validation_loss)}"
        if self.save_only_last_epoch:
            if self.current_epoch == self.trainer.max_epochs - 1:
                self.tokenizer.save_pretrained(path)
                self.model.save_pretrained(path)
        else:
            self.tokenizer.save_pretrained(path)
            self.model.save_pretrained(path)

    def validation_epoch_end(self, validation_step_outputs):
        _loss = [x.cpu() for x in validation_step_outputs]
        self.average_validation_loss = np.round(
            torch.mean(torch.stack(_loss)).item(),
            4,
        )


class SimpleT5:
    """ Custom SimpleT5 class """

    def __init__(self) -> None:
        """ initiates SimpleT5 class """
        pass

    def from_pretrained(self, model_type="t5", model_name="t5-base") -> None:
        """
        loads T5/MT5 Model model for training/finetuning
        Args:
            model_type (str, optional): "t5" or "mt5" . Defaults to "t5".
            model_name (str, optional): exact model architecture name, "t5-base" or "t5-large". Defaults to "t5-base".
        """
        if model_type == "t5":
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_name}")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(f"{model_name}")
            self.model = MT5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == "byt5":
            self.tokenizer = ByT5Tokenizer.from_pretrained(f"{model_name}")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == 'longt5':
            self.model = LongT5ForConditionalGeneration.from_pretrained(f"{model_name}")
            self.tokenizer = AutoTokenizer.from_pretrained(f"{model_name}")

    def train(
        self,
        train_df: pd.DataFrame,
        eval_df: pd.DataFrame,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
        batch_size: int = 8,
        max_epochs: int = 5,
        use_gpu: bool = True,
        outputdir: str = "outputs",
        early_stopping_patience_epochs: int = 0,  # 0 to disable early stopping feature
        precision=32,
        logger="default",
        dataloader_num_workers: int = 2,
        save_only_last_epoch: bool = False,
    ):
        """
        trains T5/MT5 model on custom dataset
        Args:
            train_df (pd.DataFrame): training datarame. Dataframe must have 2 column --> "source_text" and "target_text"
            eval_df ([type], optional): validation datarame. Dataframe must have 2 column --> "source_text" and "target_text"
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
            batch_size (int, optional): batch size. Defaults to 8.
            max_epochs (int, optional): max number of epochs. Defaults to 5.
            use_gpu (bool, optional): if True, model uses gpu for training. Defaults to True.
            outputdir (str, optional): output directory to save model checkpoints. Defaults to "outputs".
            early_stopping_patience_epochs (int, optional): monitors val_loss on epoch end and stops training, if val_loss does not improve after the specied number of epochs. set 0 to disable early stopping. Defaults to 0 (disabled)
            precision (int, optional): sets precision training - Double precision (64), full precision (32) or half precision (16). Defaults to 32.
            logger (pytorch_lightning.loggers) : any logger supported by PyTorch Lightning. Defaults to "default". If "default", pytorch lightning default logger is used.
            dataloader_num_workers (int, optional): number of workers in train/test/val dataloader
            save_only_last_epoch (bool, optional): If True, saves only the last epoch else models are saved at every epoch
        """
        self.data_module = LightningDataModule(
            train_df,
            eval_df,
            self.tokenizer,
            batch_size=batch_size,
            source_max_token_len=source_max_token_len,
            target_max_token_len=target_max_token_len,
            num_workers=dataloader_num_workers,
        )

        self.T5Model = LightningModel(
            tokenizer=self.tokenizer,
            model=self.model,
            outputdir=outputdir,
            save_only_last_epoch=save_only_last_epoch,
        )

        # add callbacks
        callbacks = [TQDMProgressBar(refresh_rate=5)]

        if early_stopping_patience_epochs > 0:
            early_stop_callback = EarlyStopping(
                monitor="val_loss",
                min_delta=0.00,
                patience=early_stopping_patience_epochs,
                verbose=True,
                mode="min",
            )
            callbacks.append(early_stop_callback)

        # add gpu support
        gpus = 1 if use_gpu else 0

        # add logger
        loggers = True if logger == "default" else logger

        # prepare trainer
        trainer = pl.Trainer(
            logger=loggers,
            callbacks=callbacks,
            max_epochs=max_epochs,
            gpus=gpus,
            precision=precision,
            log_every_n_steps=1,
        )

        # fit trainer
        trainer.fit(self.T5Model, self.data_module)

    def load_model(
        self, model_type: str = "t5", model_dir: str = "outputs", use_gpu: bool = False
    ):
        """
        loads a checkpoint for inferencing/prediction
        Args:
            model_type (str, optional): "t5" or "mt5". Defaults to "t5".
            model_dir (str, optional): path to model directory. Defaults to "outputs".
            use_gpu (bool, optional): if True, model uses gpu for inferencing/prediction. Defaults to True.
        """
        if model_type == "t5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == "mt5":
            self.model = MT5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = MT5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == "byt5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = ByT5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == 'longt5':
            self.model = LongT5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}")

        if use_gpu:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                raise "exception ---> no gpu found. set use_gpu=False, to use CPU"
        else:
            self.device = torch.device("cpu")

        self.model = self.model.to(self.device)

    def predict(
        self,
        source_text: str,
        max_length: int = 512,
        num_return_sequences: int = 1,
        num_beams: int = 2,
        top_k: int = 50,
        top_p: float = 0.95,
        do_sample: bool = True,
        repetition_penalty: float = 2.5,
        length_penalty: float = 1.0,
        early_stopping: bool = True,
        skip_special_tokens: bool = True,
        clean_up_tokenization_spaces: bool = True,
    ):
        """
        generates prediction for T5/MT5 model
        Args:
            source_text (str): any text for generating predictions
            max_length (int, optional): max token length of prediction. Defaults to 512.
            num_return_sequences (int, optional): number of predictions to be returned. Defaults to 1.
            num_beams (int, optional): number of beams. Defaults to 2.
            top_k (int, optional): Defaults to 50.
            top_p (float, optional): Defaults to 0.95.
            do_sample (bool, optional): Defaults to True.
            repetition_penalty (float, optional): Defaults to 2.5.
            length_penalty (float, optional): Defaults to 1.0.
            early_stopping (bool, optional): Defaults to True.
            skip_special_tokens (bool, optional): Defaults to True.
            clean_up_tokenization_spaces (bool, optional): Defaults to True.
        Returns:
            list[str]: returns predictions
        """
        input_ids = self.tokenizer.encode(
            source_text, return_tensors="pt", add_special_tokens=True
        )
        input_ids = input_ids.to(self.device)
        inputs_embeds = self.model.get_input_embeddings()(input_ids)
        generated_ids = self.model.generate(
            input_ids = input_ids,
            num_beams=num_beams,
            max_length=max_length,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            early_stopping=early_stopping,
            top_p=top_p,
            top_k=top_k,
            num_return_sequences=num_return_sequences,
        )
        preds = [
            self.tokenizer.decode(
                g,
                skip_special_tokens=skip_special_tokens,
                clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            )
            for g in generated_ids
        ]
        return preds


INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [10]:
model = SimpleT5()
# https://huggingface.co/google/long-t5-tglobal-xl
# https://huggingface.co/google/long-t5-tglobal-base
model.from_pretrained(model_type="t5", model_name="t5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [11]:
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=250, 
            target_max_token_len=50, 
            batch_size=2, max_epochs=5, use_gpu=True)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [12]:
df.head(1)

,source_text,target_text
0,summarize: Abstract\n\nWe model the extent to ...,The Effectiveness of Targeted Quarantine for M...


In [16]:
# let's load the trained model for inferencing:
# model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-309.1258-val-loss-339.6229", use_gpu=True)

model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-2.7354-val-loss-1.9118", use_gpu=True)
text_to_summarize="""A single dose of the COVID-19 vaccine made by either Pfizer or AstraZeneca cuts a person’s risk of transmitting SARS-CoV-2 to their closest contacts by as much as half, according to an analysis of more than 365,000 households in the United Kingdom.

Although the vaccines have been shown to reduce COVID-19 symptoms and serious illness, their ability to prevent coronavirus transmission has been unclear. Kevin Dunbar, Gavin Dabrera and their colleagues at Public Health England in London looked for cases in which someone became infected with SARS-CoV-2 after receiving a dose of either vaccine (R. J. Harris et al. Preprint at Knowledge Hub https://go.nature.com/3e3iu1i; 2021). They then assessed how often those individuals transmitted the virus to household contacts.
The team found that people who had been vaccinated for at least 21 days could still test positive for the virus. But viral transmission from these individuals to others in their households was 40–50% lower than transmission in households in which the first person to test positive had not been vaccinated. Results for the two vaccines were similar. The findings have not yet been peer reviewed.
"""
print("\nSummary\n")
model.predict(text_to_summarize)


Summary



['A single dose of the COVID-19 vaccine reduces the risk of transmission to household contacts. researchers at Public Health England in London found that people who had been vaccinated for at least 21 days could still test positive for the virus.']

In [17]:
text_to_summerize = '''
Different ways of calculating mortality ratios during epidemics can yield widely different results, particularly during the COVID-19 pandemic.
 We formulate both a survival probability model and an associated infection duration-dependent SIR model to define individual- and
  population-based estimates of dynamic mortality ratios. The key parameters that affect the dynamics of the different mortality estimates
   are the incubation period and the length of time individuals were infected before confirmation of infection. We stress that none of these
    ratios are accurately represented by the often misinterpreted case fatality ratio (CFR), the number of deaths to date divided by the total
     number of infected cases to date. Using available data on the recent SARS-CoV-2 outbreaks and simple assumptions, we estimate and compare the 
     different dynamic mortality ratios and highlight their differences. Informed by our modeling, we propose a more systematic method to 
     determine mortality ratios during epidemic outbreaks and discuss sensitivity to confounding effects and errors in the data.
'''
print("\nSummary\n")
model.predict(text_to_summerize)


Summary



['mortality ratios during epidemic outbreaks. dynamic mortality ratios during epidemic outbreaks. CFR, or the number of deaths to date divided by the total number of infected cases to date. sensitivity to confounding effects and errors in the data.']

In [18]:
text_to_summerize = '''
WHO is gathering the latest international multilingual scientific findings and knowledge on COVID-19.
 The global literature cited in the WHO COVID-19 Research Database is updated daily (Monday through Friday) from searches
  of bibliographic databases, hand searching, and the addition of other expert-referred scientific articles. This database
   represents a comprehensive multilingual source of current literature on the topic. While it may not be exhaustive, new 
   research is added regularly.
'''
print("\nSummary\n")
model.predict(text_to_summerize)


Summary



['The WHO COVID-19 Research Database is a global multilingual resource on the topic.']

In [19]:
# let's load the trained model for inferencing:
# model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-309.1258-val-loss-339.6229", use_gpu=True)

model.load_model("t5","/content/outputs/simplet5-epoch-1-train-loss-1.9319-val-loss-2.1405", use_gpu=True)
text_to_summarize="""A single dose of the COVID-19 vaccine made by either Pfizer or AstraZeneca cuts a person’s risk of transmitting SARS-CoV-2 to their closest contacts by as much as half, according to an analysis of more than 365,000 households in the United Kingdom.

Although the vaccines have been shown to reduce COVID-19 symptoms and serious illness, their ability to prevent coronavirus transmission has been unclear. Kevin Dunbar, Gavin Dabrera and their colleagues at Public Health England in London looked for cases in which someone became infected with SARS-CoV-2 after receiving a dose of either vaccine (R. J. Harris et al. Preprint at Knowledge Hub https://go.nature.com/3e3iu1i; 2021). They then assessed how often those individuals transmitted the virus to household contacts.
The team found that people who had been vaccinated for at least 21 days could still test positive for the virus. But viral transmission from these individuals to others in their households was 40–50% lower than transmission in households in which the first person to test positive had not been vaccinated. Results for the two vaccines were similar. The findings have not yet been peer reviewed.
"""
print("\nSummary\n")
model.predict(text_to_summarize)


Summary



['Public Health England in London. a single dose of the COVID-19 vaccine. two individuals tested positive for SARS-CoV-2.,. researchers.']

In [20]:
# let's load the trained model for inferencing:
# model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-309.1258-val-loss-339.6229", use_gpu=True)

model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-2.6769-val-loss-2.1557", use_gpu=True)
text_to_summarize="""A single dose of the COVID-19 vaccine made by either Pfizer or AstraZeneca cuts a person’s risk of transmitting SARS-CoV-2 to their closest contacts by as much as half, according to an analysis of more than 365,000 households in the United Kingdom.

Although the vaccines have been shown to reduce COVID-19 symptoms and serious illness, their ability to prevent coronavirus transmission has been unclear. Kevin Dunbar, Gavin Dabrera and their colleagues at Public Health England in London looked for cases in which someone became infected with SARS-CoV-2 after receiving a dose of either vaccine (R. J. Harris et al. Preprint at Knowledge Hub https://go.nature.com/3e3iu1i; 2021). They then assessed how often those individuals transmitted the virus to household contacts.
The team found that people who had been vaccinated for at least 21 days could still test positive for the virus. But viral transmission from these individuals to others in their households was 40–50% lower than transmission in households in which the first person to test positive had not been vaccinated. Results for the two vaccines were similar. The findings have not yet been peer reviewed.
"""
print("\nSummary\n")
model.predict(text_to_summarize)


Summary



['a single dose of the COVID-19 vaccine. researchers at Public Health England in London and their results were published online today., or.s....']

In [21]:
# let's load the trained model for inferencing:
# model.load_model("t5","/content/outputs/simplet5-epoch-0-train-loss-309.1258-val-loss-339.6229", use_gpu=True)

model.load_model("t5","/content/outputs/simplet5-epoch-3-train-loss-1.1488-val-loss-2.3041", use_gpu=True)
text_to_summarize="""A single dose of the COVID-19 vaccine made by either Pfizer or AstraZeneca cuts a person’s risk of transmitting SARS-CoV-2 to their closest contacts by as much as half, according to an analysis of more than 365,000 households in the United Kingdom.

Although the vaccines have been shown to reduce COVID-19 symptoms and serious illness, their ability to prevent coronavirus transmission has been unclear. Kevin Dunbar, Gavin Dabrera and their colleagues at Public Health England in London looked for cases in which someone became infected with SARS-CoV-2 after receiving a dose of either vaccine (R. J. Harris et al. Preprint at Knowledge Hub https://go.nature.com/3e3iu1i; 2021). They then assessed how often those individuals transmitted the virus to household contacts.
The team found that people who had been vaccinated for at least 21 days could still test positive for the virus. But viral transmission from these individuals to others in their households was 40–50% lower than transmission in households in which the first person to test positive had not been vaccinated. Results for the two vaccines were similar. The findings have not yet been peer reviewed.
"""
print("\nSummary\n")
model.predict(text_to_summarize)


Summary



['A single dose of the COVID-19 vaccine reduces the risk of transmission to household contacts by as much as half, according to a new study.']